# Set random seeds

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

# Imports

In [ ]:
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
from utils.model_utils import (
    separate_features,
    split_data,
    get_ann_random_hyperparameters,
    get_evaluation_metrics,
    get_optimal_hyperparameters,
)

# Prepare data

## Load data

In [ ]:
raw_df = gpd.read_file("datasets/5_split/df_fe.gpkg")

In [ ]:
df = raw_df.copy()

## Separate features

In [ ]:
labels, outer_fold_ids, outer_splits, inner_fold_ids, inner_splits, features = (
    separate_features(df)
)
predictor_cols = ["greenspace_proportion", "imd", "f_m_ratio", "mean_age"]

# Model functions

## Build model

In [ ]:
def build_model(train_features, no_of_layers, no_of_nodes, learning_rate):

    layers = []

    normaliser = keras.layers.Normalization(axis=-1)
    normaliser.adapt(np.array(train_features))
    layers.append(normaliser)

    for layer_no in range(no_of_layers):
        layers.append(keras.layers.Dense(no_of_nodes[layer_no], activation="relu"))

    layers.append(keras.layers.Dense(1))  # Single output for regression value

    model = keras.Sequential(layers)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse"
    )

    return model

## Build early stopper

In [ ]:
def build_early_stopper():
    early_stopper = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
    return early_stopper

# Evaluate model

## Initialise results array

In [ ]:
outer_cv_results = []

##  Evaluate using nested cross-validation loop

In [ ]:
for current_outer_split in outer_splits:

    hp_combinations = []
    cv_results = []

    # Get training and validation sets for current outer split
    (
        outer_train_features,
        outer_train_labels,
        outer_val_features,
        outer_val_labels,
        current_inner_fold_ids,
    ) = split_data(
        current_outer_split,
        outer_fold_ids,
        features,
        labels,
        is_outer=True,
        inner_fold_ids=inner_fold_ids,
    )

    # Loop to test 10 hyperparameter combinations
    for i in range(8):

        # Get hyperparameters
        no_of_layers, no_of_nodes, learning_rate = get_ann_random_hyperparameters(
            features
        )
        current_hps = {
            "outer_loop_split": current_outer_split,
            "no_of_layers": no_of_layers,
            "no_of_nodes": no_of_nodes,
            "learning_rate": learning_rate,
        }
        hp_combinations.append(current_hps)

        # Inner cross-validation for model selection
        for current_inner_split in inner_splits:
            print(
                f"\n --- Outer split {current_outer_split}: Training model {i} on inner split {current_inner_split} ---"
            )

            # Get training and validation sets for current inner split
            (
                inner_train_features,
                inner_train_labels,
                inner_val_features,
                inner_val_labels,
                _,
            ) = split_data(
                current_inner_split,
                current_inner_fold_ids[
                    f"inner_loop_{current_inner_split + 1}_fold_id_python"
                ],
                outer_train_features,
                outer_train_labels,
            )

            # Build model
            model = build_model(
                inner_train_features, no_of_layers, no_of_nodes, learning_rate
            )
            early_stopper = build_early_stopper()

            # Fit model
            model.fit(
                inner_train_features,
                inner_train_labels,
                epochs=200,
                validation_data=0.2,
                callbacks=[early_stopper],
                verbose=1,
            )

            # Get predictions using fitted model
            predictions = model.predict(inner_val_features).flatten()

            # Get accuracy scores
            mae, mse, r2 = get_evaluation_metrics(
                inner_val_features, inner_val_labels, predictions
            )

            # Add scores for current fold to results
            cv_results.append(
                {
                    "hp_combination": i,
                    "inner_split": current_inner_split,
                    "hps": current_hps,
                    "mae": mae,
                    "mse": mse,
                    "r2": r2,
                }
            )

    # Get optimal hyperparameters for current outer split training set
    opt_hps = get_optimal_hyperparameters(hp_combinations, cv_results)
    opt_no_of_layers = opt_hps["no_of_layers"]
    opt_no_of_nodes = opt_hps["no_of_nodes"]
    opt_learning_rate = opt_hps["learning_rate"]

    # Build model
    model = build_model(
        outer_train_features, opt_no_of_layers, opt_no_of_nodes, opt_learning_rate
    )
    early_stopper = build_early_stopper()

    # Fit model
    model.fit(
        outer_train_features,
        outer_train_labels,
        epochs=200,
        validation_split = 0.2,
        callbacks=[early_stopper],
        verbose=1,
    )

    # Get predictions using fitted model
    predictions = model.predict(outer_val_features).flatten()

    # Get accuracy scores
    mae, mse, r2 = get_evaluation_metrics(
        outer_val_features, outer_val_labels, predictions
    )

    # Add scores for current fold to results
    outer_cv_results.append(
        {
            "outer_split": current_outer_split,
            "hps": opt_hps,
            "mae": mae,
            "mse": mse,
            "r2": r2,
            "inner_cv_results": cv_results,
        }
    )

# Save output

In [ ]:
with open("outputs/model_results/ann_fe.pkl", "wb") as f:
    pickle.dump(outer_cv_results, f)